In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torch.nn.functional as F

from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import MultiplicativeLR

In [2]:
torch.manual_seed(42)

In [14]:
df = pd.read_csv("/kaggle/input/ferrrr/fer2013.csv")

In [49]:
def load_fer2013(df):
    pixels = df['pixels'].tolist()
    width, height = 48, 48
    faces = [np.asarray([int(pixel) for pixel in face.split(' ')], dtype=np.uint8).reshape(1, width, height) for face in pixels]
    faces = np.array(faces) / 255.0
    emotions = np.array(df['emotion'])

    faces = torch.tensor(faces, dtype=torch.float32)
    emotions = torch.tensor(emotions, dtype=torch.long)
    dataset = TensorDataset(faces, emotions)
    
    generator = torch.Generator().manual_seed(42)
    return random_split(dataset, [0.8, 0.1, 0.1], generator=generator)

In [50]:
train_dataset, val_dataset, test_dataset = load_fer2013(df)

In [55]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [206]:
class FERModel(nn.Module):
    def __init__(self, nb_classes: int):
        super(FERModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, nb_classes)

        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.4)

    
    def forward(self, x):
        x = F.gelu(self.conv1(x))
        
        x = self.conv2(x)
        x = F.gelu(F.max_pool2d(x, 2))
        x = self.batch_norm1(x)
        
        x = self.conv3(x)
        x = F.gelu(F.max_pool2d(x, 2))
        x = self.batch_norm2(x)

        x = self.dropout1(x)

        x = F.gelu(self.conv4(x))
        x = self.batch_norm3(x)
        
        x = x.view(x.size(0), -1)
        x = F.gelu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return x

In [241]:
EPOCHS = 50
NB_CLASSES = 7

In [242]:
model = FERModel(NB_CLASSES).cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.97)

In [243]:
for epoch in range(EPOCHS):
    l, c = 0, 0
    model.train()
    for data, labels in train_dataloader:
        data, labels = data.cuda(), labels.cuda()
        outputs = model(data)
        
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        
        l += loss.item()
        c += len(data)
        
        loss.backward()
        optimizer.step()        

    v_l, v_c = 0, 0
    model.eval()
    with torch.no_grad():
        for data, labels in val_dataloader:
            data, labels = data.cuda(), labels.cuda()
            outputs = model(data)
            
            loss = criterion(outputs, labels)
            
            v_l += loss.item()
            v_c += len(data)
            
    print(f"Epoch {epoch + 1} - train_loss: {l / c} - val_loss: {v_l / v_c} - lr: {scheduler.get_last_lr()[0]}")
    scheduler.step()

Epoch 1 - train_loss: 1.5969816377294972 - val_loss: 1.5144949833574186 - lr: 0.0003
Epoch 2 - train_loss: 1.3801115771446109 - val_loss: 1.3641658469673383 - lr: 0.00029099999999999997
Epoch 3 - train_loss: 1.2448184197723804 - val_loss: 1.256703098028209 - lr: 0.00028226999999999994
Epoch 4 - train_loss: 1.1293694969504107 - val_loss: 1.287656192295928 - lr: 0.00027380189999999993
Epoch 5 - train_loss: 0.9885922453537174 - val_loss: 1.1862979211592017 - lr: 0.0002655878429999999
Epoch 6 - train_loss: 0.8596524503329871 - val_loss: 1.1490083536619462 - lr: 0.0002576202077099999
Epoch 7 - train_loss: 0.7217759128731329 - val_loss: 1.2053549299667923 - lr: 0.0002498916014786999
Epoch 8 - train_loss: 0.5984427245142401 - val_loss: 1.211937308610399 - lr: 0.00024239485343433888
Epoch 9 - train_loss: 0.49300789177023296 - val_loss: 1.2136511097182867 - lr: 0.0002351230078313087
Epoch 10 - train_loss: 0.40026038089142324 - val_loss: 1.3741643282323537 - lr: 0.00022806931759636944
Epoch 11 -

In [246]:
torch.save(model.state_dict(), "weights.pt")

In [244]:
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [245]:
model.eval()
s = 0

for data, labels in test_dataloader:
    data, labels = data.cuda(), labels.cuda()
    outputs = model(data)
    _, prediction = torch.max(outputs, 1)

    s += (prediction == labels).sum().item()

print(f"Test accuracy: {s / len(test_dataset)}")

Test accuracy: 0.6073021181716833
